# Import de pacotes

In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

import random
from pprint import pprint


# Funções para treinar uma árvore de decisão

Importante! A variável resposta deve ser a última coluna do data frame.

## Funções auxiliares

### Função para separar dados em treino e teste

In [ ]:
def train_test_split(df, test_size):
    
    if isinstance(test_size, float):
        test_size = round(test_size * len(df))
    
    indices = df.index.tolist()
    test_indices = random.sample(population = indices, k = test_size)
    test_df = df.loc[test_indices]
    train_df = df.drop(test_indices)
    
    return train_df, test_df

### Dados com só uma resposta?

In [10]:
def check_purity(data):
    label_column = data[:,-1]
    unique_classes = np.unique(label_column)

    if len(unique_classes) == 1:
        return True
    else: 
        return False

### Classificador

In [11]:
def classify_data(data):
    
    label_column = data[:,-1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts = True)
    
    index = counts_unique_classes.argmax()
    
    classification = unique_classes[index]
    
    return classification

### Potencial split

In [12]:
def get_potential_splits(data):
    
    potential_splits = {}
    _, n_columns = data.shape
    for column_index in range(n_columns - 1):
        potential_splits[column_index] = []
        values = data[:, column_index]
        unique_values = np.unique(values)
        
        for index in range(len(unique_values)):
            if index != 0:
                current_value = unique_values[index]
                previous_value = unique_values[index - 1]
                potential_split = (current_value + previous_value) / 2
                
                potential_splits[column_index].append(potential_split)
                
    return potential_splits

### Split dos dados

In [13]:
def split_data(data, split_column, split_value):
    
    split_column_values = data[:, split_column]
    
    data_below = data[split_column_values <= split_value]
    data_above = data[split_column_values > split_value]
    
    return data_below, data_above

### Menor impureza. 

Foram criadas as funções podendo ser usadas as medidas de impureza Gini ou entropia.

In [14]:
def calculate_entropy(data):
    
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts = True)

    probabilities = counts / counts.sum()
    entropy = sum(probabilities * -np.log2(probabilities))
    return entropy

In [15]:
def calculate_gini(data):
    label_column = data[:, -1]
    _, counts = np.unique(label_column, return_counts = True)
    
    probabilities = (counts / counts.sum())**2
    
    gini = 1 - sum(probabilities)
    
    return gini

In [16]:
def calculate_overall_entropy(data_below, data_above):
    n_data_points = len(data_below) + len(data_above)
    
    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points
    
    overall_entropy = (p_data_below * calculate_entropy(data_below)) + (p_data_above * calculate_entropy(data_above))
    
    return overall_entropy

In [17]:
def calculate_overall_gini(data_below, data_above):
    n_data_points = len(data_below) + len(data_above)
    
    p_data_below = len(data_below) / n_data_points
    p_data_above = len(data_above) / n_data_points
    
    overall_gini = (p_data_below * calculate_gini(data_below)) + (p_data_above * calculate_gini(data_above))
    
    return overall_gini

### Melhor split

In [18]:
def determine_best_split(data, potential_splits, impurity = 'gini'):
    
    overall_impurity = 999
    for column_index in potential_splits:
        for value in potential_splits[column_index]:
            data_below, data_above = split_data(data, split_column = column_index, split_value = value)
            if impurity == 'entropy':
                current_overall_impurity = calculate_overall_entropy(data_below, data_above)
            else: # impurity == 'gini'
                current_overall_impurity = calculate_overall_gini(data_below, data_above)
            
            if current_overall_impurity < overall_impurity:
                overall_impurity = current_overall_impurity
                best_split_column = column_index
                best_split_value = value
    
    return best_split_column, best_split_value

## Função principal 

Parâmetros da função:
 - df: conjunto de dados
 - min_samples: quantidade máxima de observações nas folhas (finais)
 - max_depth: quantidade máxima de ramificações (splits)
 - impurity: qual a medida de impureza usada

In [21]:
def decision_tree_algorithm(df, counter = 0, min_samples = 2,max_depth = 5, impurity = 'gini'):
    
    # 
    if counter == 0:
        global column_headers
        column_headers = df.columns
        data = df.values
    else:
        data = df
        
    #base case
    if (check_purity(data)) or (len(data) < min_samples) or (counter == max_depth):
        classification = classify_data(data)
        return classification
    
    #recursive part
    else:
        counter += 1
        
        potential_splits = get_potential_splits(data)
        split_column, split_value = determine_best_split(data, potential_splits, impurity = 'gini')
        data_below, data_above = split_data(data,split_column, split_value)
        
        # instantiate sub-tree
        feature_name = column_headers[split_column]
        question = "{} <= {:.6f}".format(feature_name, split_value)
        sub_tree = {question: []}
    
        #find answers (recursion)
        yes_answer = decision_tree_algorithm(data_below, counter, min_samples, impurity, max_depth)
        no_answer = decision_tree_algorithm(data_above, counter, min_samples, impurity, max_depth)
        
        if yes_answer == no_answer:
            sub_tree = yes_answer
        else:
            sub_tree[question].append(yes_answer)
            sub_tree[question].append(no_answer)
        
    return sub_tree

# Funções para classificação de novos dados

## Classificação

In [25]:
sub_tree = {question: [yes_answer, no_answer]}

array(['M'], dtype=object)

In [ ]:
def classify_example(example, tree):
    
    question = list(tree.keys())[0]
    feature_name, comparitson_operator,value = question.split()
    
    if example[feature_name] <= float(value):
        answer = tree[question][0]
    else: 
        answer = tree[question][1]
        
    if not isinstance(answer, dict):
        return answer
    else:
        residual_tree = answer
        return classify_example(example, answer)

## Acurácia

In [ ]:
def calculate_accuracy(df, tree):
    
    df['classification'] = df.apply(classify_example, axis=1, args = (tree,))
    df['classification_correct'] = df.classification == df.label
    
    accuracy = df.classification_correct.mean()
    
    return accuracy

# Conjunto de dados

## Dicionário das variáveis:

 **Temperature (K)}**:

**Luminosity(L/Lo)**:

**Radius(R/Ro)**:

**Absolute magnitude(Mv)**:

**Star Type**: Brown Dwarf = 0, Red Dwarf = 1, White Dwarf = 2, Main Sequence = 3, Supergiant = 4, Hypergiant = 5

**Star color**:

**Spectral Class**:

The Luminosity and radius of each star is calculated w.r.t. that of the values of Sun.

Lo = 3.828 x 10^26 Watts

Ro = 6.9551 x 10^8 m

## Carregamento dos dados

In [3]:
dados = pd.read_csv('stars.csv')

In [4]:
dados.head()

,Temperature (K),Luminosity(L/Lo),Radius(R/Ro),Absolute magnitude(Mv),Star type,Star color,Spectral Class
0,3068,0.002400,0.1700,16.12,0,Red,M
1,3042,0.000500,0.1542,16.60,0,Red,M
2,2600,0.000300,0.1020,18.70,0,Red,M
3,2800,0.000200,0.1600,16.65,0,Red,M
4,1939,0.000138,0.1030,20.06,0,Red,M


In [5]:
dados.shape

(240, 7)

In [6]:
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 240 entries, 0 to 239
Data columns (total 7 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Temperature (K)         240 non-null    int64  
 1   Luminosity(L/Lo)        240 non-null    float64
 2   Radius(R/Ro)            240 non-null    float64
 3   Absolute magnitude(Mv)  240 non-null    float64
 4   Star type               240 non-null    int64  
 5   Star color              240 non-null    object 
 6   Spectral Class          240 non-null    object 
dtypes: float64(3), int64(2), object(2)
memory usage: 13.2+ KB


## Preparando os dados

### Variável resposta (alvo)

In [ ]:
dados['Star type'].unique()

In [ ]:
dados['label'] = dados['Star type']
dados = dados.drop(['Star type'], axis = 1)

In [ ]:
dados.info()

### Separando os dados entre teste e treino

In [ ]:
random.seed(0)
treino, teste = train_test_split(dados,0.1)

In [ ]:
treino.shape, teste.shape

# Treinando uma árvore de decisão

In [22]:
tree = decision_tree_algorithm(dados.drop('Star color', axis = 1))

In [23]:
pprint(tree)

{'Temperature (K) <= 3924.500000': ['M',
                                    {'Absolute magnitude(Mv) <= -4.280000': [{'Radius(R/Ro) <= 403.450000': [{'Temperature (K) <= 32880.000000': ['O',
                                                                                                                                                  {'Temperature (K) <= 33970.000000': [{'Luminosity(L/Lo) <= 296000.000000': ['B',
                                                                                                                                                                                                                              'O']},
                                                                                                                                                                                       'O']}]},
                                                                                                             {'Temperature (K) <= 29289.000000': [{'Temperature 